In [1]:
import numpy as np
import pandas as pd
import sys
import json
import math
from sklearn import svm
from sklearn.model_selection import train_test_split
from sktime.classification.compose import TimeSeriesForestClassifier
from sktime.utils.slope_and_trend import _slope
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from hmmlearn.hmm import GaussianHMM
from sktime.classification.all import TimeSeriesForest
from sktime.transformations.panel.compose import ColumnConcatenator
from sklearn.preprocessing import MinMaxScaler

/home/anli5005/miniconda3/envs/r/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [5]:
df = pd.read_csv("data2.csv")

In [3]:
def construct_model():
    steps = [
        ("concatenate", ColumnConcatenator()),
        ("classify", TimeSeriesForest()),
    ]
    return Pipeline(steps, verbose=1)

In [4]:
cols = ["class", "tsa", "visits_hotels", "visits_restaurants", "visits_retail", "visits_personal_care", "visits_gyms", "visits_health", "temperature", "flu", "tests", "hospitalized", "svi", "density"]

In [6]:
y_series = df["class"]
X_series = df.copy()
wind = 14
min_max_scaler = MinMaxScaler()
for col in cols:
    X_series[[col]] = min_max_scaler.fit_transform(X_series[[col]])
X = np.empty((len(df) - wind, wind, len(cols)))
include_set = []
for i in range(len(df) - wind):
    X[i] = np.array(X_series[cols][i:i+wind])
    include_set.append(len(X_series["state"][i:i+wind].unique()) == 1)

X = X[include_set]

y = np.array(y_series[wind:])
y = y[include_set]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [8]:
from sklearn.metrics import f1_score, balanced_accuracy_score, make_scorer, plot_roc_curve
f1_scorer = make_scorer(f1_score, average="micro")

In [9]:
models = []
scores = []
for i in range(10):
    model = construct_model()
    model.fit(X_train, y_train)
    models.append(model)

[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.8s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 8.1min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.8s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 8.1min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.8s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 8.1min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.8s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 7.6min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.6s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 6.5min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.6s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 6.4min
[Pipeline] ....... (step 1 of 2) Processing concatenate, total=   0.6s
[Pipeline] .......... (step 2 of 2) Processing classify, total= 6.5min
[Pipel

In [11]:
for i in range(10):
    print(f1_scorer(models[i], X_test, y_test))

0.7857142857142857
0.7905328798185941
0.7857142857142857
0.7939342403628118
0.7865646258503401
0.7899659863945578
0.7803287981859411
0.7936507936507936
0.7893990929705215
0.7851473922902494


In [ ]:
y

In [13]:
np.mean([float(x) for x in """0.7857142857142857
0.7905328798185941
0.7857142857142857
0.7939342403628118
0.7865646258503401
0.7899659863945578
0.7803287981859411
0.7936507936507936
0.7893990929705215
0.7851473922902494""".split("\n")])

0.7880952380952381

In [8]:
clf.score(X, y)

0.9763305222875771

In [172]:
from sktime.datasets import load_arrow_head
X, y = load_arrow_head(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [173]:
y_test.shape

(53,)

In [52]:
# Random Forest: 0.9489230769230769
# RISE: 0.7989331144850226